# Using environments


## Contents

1. [Introduction](#Introduction)
1. [Setup](#Setup)
1. [Use curated environment](#Use-curated-environment)
1. [Create environment](#Create-environment)
    1. Add Python packages
    1. Specify environment variables
1. [Submit run using environment](#Submit-run-using-environment)
1. [Register environment](#Register-environment)
1. [List and get existing environments](#List-and-get-existing-environments)
1. [Other ways to create environments](#Other-ways-to-create-environments)
    1. From existing Conda environment
    1. From Conda or pip files
1. [Using environments for inferencing](#Using-environments-for-inferencing)
1. [Docker settings](#Docker-settings)
1. [Spark and Azure Databricks settings](#Spark-and-Azure-Databricks-settings)
1. [Next steps](#Next-steps)

## Introduction

Azure ML environments are an encapsulation of the environment where your machine learning training happens. They define Python packages, environment variables, Docker settings and other attributes in declarative fashion. Environments are versioned: you can update them and retrieve old versions to revisit and review your work.

Environments allow you to:
* Encapsulate dependencies of your training process, such as Python packages and their versions.
* Reproduce the Python environment on your local computer in a remote run on VM or ML Compute cluster
* Reproduce your experimentation environment in production setting.
* Revisit and audit the environment in which an existing model was trained.

Environment, compute target and training script together form run configuration: the full specification of training run.

## Setup

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you go through the [configuration notebook](../../../configuration.ipynb) first if you haven't.

First, let's validate Azure ML SDK version and connect to workspace.

In [1]:
import azureml.core
print(azureml.core.VERSION)

1.20.0


In [2]:
from azureml.core.workspace import Workspace
ws = Workspace.from_config()
ws.get_details()

{'id': '/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourceGroups/AMLworkshop-rg/providers/Microsoft.MachineLearningServices/workspaces/AMLworkshop',
 'name': 'AMLworkshop',
 'identity': {'principal_id': 'ec3d3147-5547-4237-8ab7-25f145c914b9',
  'tenant_id': '72f988bf-86f1-41af-91ab-2d7cd011db47',
  'type': 'SystemAssigned'},
 'location': 'westeurope',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'sku': 'Basic',
 'workspaceid': '0e272668-09cc-42d0-958d-694cc918abf6',
 'sdkTelemetryAppInsightsKey': '6d7b2b55-6d18-4126-a3ee-251f78cfa6e5',
 'description': '',
 'friendlyName': 'AMLworkshop',
 'creationTime': '2020-11-16T10:35:26.1010460+00:00',
 'containerRegistry': '/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourceGroups/AMLworkshop-rg/providers/Microsoft.ContainerRegistry/registries/0e27266809cc42d0958d694cc918abf6',
 'keyVault': '/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourcegroups/amlworkshop-rg/providers/microsoft.keyvault/va

## Use curated environments

Curated environments are provided by Azure Machine Learning and are available in your workspace by default. They contain collections of Python packages and settings to help you get started different machine learning frameworks. 

  * The __AzureML-Minimal__ environment contains a minimal set of packages to enable run tracking and asset uploading. You can use it as a starting point for your own environment.
  * The __AzureML-Tutorial__ environment contains common data science packages, such as Scikit-Learn, Pandas and Matplotlib, and larger set of azureml-sdk packages.
 
Curated environments are backed by cached Docker images, reducing the run preparation cost.
 
You can get a curated environment using

In [3]:
from azureml.core import Environment

curated_env = Environment.get(workspace=ws, name="AzureML-Minimal")

To list curated environments, use following code.

**Note**: The name prefixes _AzureML_ and _Microsoft_ are reserved for curated environments. Do not use them for your own environments

In [4]:
envs = Environment.list(workspace=ws)

for env in envs:
    if env.startswith("AzureML"):
        print("Name",env)
        print("packages", envs[env].python.conda_dependencies.serialize_to_string())

Name AzureML-AutoML
packages channels:
- anaconda
- conda-forge
- pytorch
dependencies:
- python=3.6.2
- pip=20.2.4
- pip:
  - azureml-core==1.20.0
  - azureml-pipeline-core==1.20.0
  - azureml-telemetry==1.20.0
  - azureml-defaults==1.20.0
  - azureml-interpret==1.20.0
  - azureml-automl-core==1.20.0
  - azureml-automl-runtime==1.20.0
  - azureml-train-automl-client==1.20.0
  - azureml-train-automl-runtime==1.20.0.post1
  - azureml-dataset-runtime==1.20.0
  - inference-schema
  - py-cpuinfo==5.0.0
  - boto3==1.15.18
  - botocore==1.18.18
- numpy~=1.18.0
- scikit-learn==0.22.1
- pandas~=0.25.0
- py-xgboost<=0.90
- fbprophet==0.5
- holidays==0.9.11
- setuptools-git
- psutil>5.0.0,<6.0.0
name: azureml_265db83b0c6014ce472c5de2f0b97e04

Name AzureML-PyTorch-1.0-GPU
packages channels:
- conda-forge
dependencies:
- python=3.6.2
- pip:
  - azureml-core==1.19.0
  - azureml-defaults==1.19.0
  - azureml-telemetry==1.19.0
  - azureml-train-restclients-hyperdrive==1.19.0
  - azureml-train-core==1.

## Create your own environment

You can create an environment by instantiating ```Environment``` object and then setting its attributes: set of Python packages, environment variables and others.

### Add Python packages

The recommended way is to specify Conda packages, as they typically come with complete set of pre-built binaries.

In [5]:
from azureml.core.environment import CondaDependencies

myenv = Environment(name="myenvserge")
conda_dep = CondaDependencies()
conda_dep.add_conda_package("scikit-learn")

You can also add pip packages, and specify the version of package

In [6]:
conda_dep.add_pip_package("pillow==5.4.1")
myenv.python.conda_dependencies=conda_dep

### Specify environment variables

You can add environment variables to your environment. These then become available using ```os.environ.get``` in your training script.

In [7]:
myenv.environment_variables = {"MESSAGE":"Hello from Azure Machine Learning"}

## Submit run using environment

When you submit a run, you can specify which environment to use. 

On the first run in given environment, Azure ML spends some time building the environment. On the subsequent runs, Azure ML keeps track of changes and uses the existing environment, resulting in faster run completion.

In [8]:
from azureml.core import ScriptRunConfig, Experiment

myexp = Experiment(workspace=ws, name = "environment-example")

To submit a run, create a run configuration that combines the script file and environment, and pass it to ```Experiment.submit```. In this example, the script is submitted to local computer, but you can specify other compute targets such as remote clusters as well.

In [13]:
src = ScriptRunConfig(source_directory=".",
                      script="hello.py",
                      compute_target="local",
                      environment=myenv)

run = myexp.submit(config=src)

run.wait_for_completion(show_output=True)

RunId: environment-example_1611045471_29f3a98d
Web View: https://ml.azure.com/experiments/environment-example/runs/environment-example_1611045471_29f3a98d?wsid=/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourcegroups/amlworkshop-rg/workspaces/amlworkshop

Streaming azureml-logs/70_driver_log.txt

[2021-01-19T08:37:55.191661] Entering context manager injector.
[context_manager_injector.py] Command line Options: Namespace(inject=['ProjectPythonPath:context_managers.ProjectPythonPath', 'RunHistory:context_managers.RunHistory', 'TrackUserError:context_managers.TrackUserError', 'UserExceptions:context_managers.UserExceptions'], invocation=['hello.py'])
Script type = None
Starting the daemon thread to refresh tokens in background for process with pid = 14357
Entering Run History Context Manager.
[2021-01-19T08:37:56.604683] Current directory: /tmp/azureml_runs/environment-example_1611045471_29f3a98d
[2021-01-19T08:37:56.604743] Preparing to call script [hello.py] with arguments:[]


{'runId': 'environment-example_1611045471_29f3a98d',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2021-01-19T08:37:54.950648Z',
 'endTimeUtc': '2021-01-19T08:37:58.851315Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': 'e2b3fde4-a75e-42f1-9baf-e0c2954938f2'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'hello.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'priority': None,
  'credentialPassthrough': False,
  'environment': {'name': 'myenvserge',
   'version': 'Autosave_2021-01-19T08:36:40Z_61b60f4b',
   'python': {'interpreterPath': 'python',
    'userManagedDependencies': False,
    'condaDependencies': {'channels': ['anaconda', 'conda-forge'],
   

To audit the environment used by for a run, you can use ```get_environment```.

In [14]:
run.get_environment()

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:20210104.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "MESSAGE": "Hello from Azure Machine Learning"
    },
    "inferencingStackVersion": null,
    "name": "myenvserge",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "co

## Register environment

You can manage environments by registering them. This allows you to track their versions, and reuse them in future runs. For example, once you've constructed an environment that meets your requirements, you can register it and use it in other experiments so as to standardize your workflow.

If you register the environment with same name, the version number is increased by one. Note that Azure ML keeps track of differences between the version, so if you re-register an identical version, the version number is not increased.

In [11]:
myenv.register(workspace=ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:20210104.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "MESSAGE": "Hello from Azure Machine Learning"
    },
    "inferencingStackVersion": null,
    "name": "myenvserge",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "co

## List and get existing environments

Your workspace contains a dictionary of registered environments. You can then use ```Environment.get``` to retrieve a specific environment with specific version.

In [12]:
for name,env in ws.environments.items():
    print("Name {} \t version {}".format(name,env.version))

restored_environment = Environment.get(workspace=ws,name="myenv",version="1")

print("Attributes of restored environment")
restored_environment

Name tutorial-env 	 version 48
Name myenvserge 	 version 2
Name AzureML-AutoML 	 version 52
Name AzureML-PyTorch-1.0-GPU 	 version 44
Name AzureML-Scikit-learn-0.20.3 	 version 43
Name AzureML-TensorFlow-1.12-CPU 	 version 42
Name AzureML-PyTorch-1.2-GPU 	 version 44
Name AzureML-TensorFlow-2.0-GPU 	 version 42
Name AzureML-TensorFlow-2.0-CPU 	 version 41
Name AzureML-Chainer-5.1.0-GPU 	 version 44
Name AzureML-TensorFlow-1.13-CPU 	 version 43
Name AzureML-Minimal 	 version 44
Name AzureML-Chainer-5.1.0-CPU 	 version 43
Name AzureML-PyTorch-1.4-GPU 	 version 34
Name AzureML-PySpark-MmlSpark-0.15 	 version 39
Name AzureML-PyTorch-1.3-CPU 	 version 39
Name AzureML-PyTorch-1.1-GPU 	 version 44
Name AzureML-TensorFlow-1.10-GPU 	 version 44
Name AzureML-PyTorch-1.2-CPU 	 version 43
Name AzureML-TensorFlow-1.13-GPU 	 version 44
Name AzureML-TensorFlow-1.10-CPU 	 version 43
Name AzureML-PyTorch-1.3-GPU 	 version 41
Name AzureML-Tutorial 	 version 60
Name AzureML-PyTorch-1.0-CPU 	 version 43
N

Exception: Error retrieving the environment definition. Code: 404
: {
  "error": {
    "code": "UserError",
    "severity": null,
    "message": "No definition exists for environment myenv version 1",
    "messageFormat": null,
    "messageParameters": null,
    "referenceCode": null,
    "detailsUri": null,
    "target": null,
    "details": [],
    "innerError": {
      "code": "NotFoundError",
      "innerError": null
    },
    "debugInfo": null
  },
  "correlation": {
    "operation": "cfe70129ece808488a38f630ff45e6d8",
    "request": "332b57a35624a84d"
  },
  "environment": "westeurope",
  "location": "westeurope",
  "time": "2021-01-19T08:36:57.3040894+00:00",
  "componentName": "environment-management"
}

## Other ways to create environments

### From existing Conda environment

You can create an environment from existing conda environment. This make it easy to reuse your local interactive environment in Azure ML remote runs. For example, if you've created conda environment using
```
conda create -n mycondaenv
```
you can create Azure ML environment out of that conda environment using
```
myenv = Environment.from_existing_conda_environment(name="myenv",conda_environment_name="mycondaenv")
```

### From conda or pip files

You can create environments from conda specification or pip requirements files using
```
myenv = Environment.from_conda_specification(name="myenv", file_path="path-to-conda-specification-file")

myenv = Environment.from_pip_requirements(name="myenv", file_path="path-to-pip-requirements-file")
```


## Using environments for inferencing

You can re-use the training environment when you deploy your model as a web service, by specifying inferencing stack version, and adding then environment to ```InferenceConfig```.

```
from azureml.core.model import InferenceConfig

myenv.inferencing_stack_version = "latest"

inference_config = InferenceConfig(entry_script="score.py", environment=myenv)
```

See [Register Model and deploy as Webservice Notebook](../../deployment/deploy-to-cloud/model-register-and-deploy.ipynb) for an end-to-end example of web service deployment.

## Docker settings

Docker container provides an efficient way to encapsulate the dependencies. When you enable Docker, Azure ML builds a Docker image and creates a Python environment within that container, given your specifications. The Docker images are reused: the first run in a new environment typically takes longer as the image is build.

**Note:** For runs on local computer or attached virtual machine, that computer must have Docker installed and enabled. Machine Learning Compute has Docker pre-installed.

Attribute ```docker.enabled``` controls whether to use Docker container or host OS for execution. 

In [ ]:
myenv.docker.enabled = True

You can specify custom Docker base image and registry. This allows you to customize and control in detail the guest OS in which your training run executes. whether to use GPU, whether to use shared volumes, and shm size.

In [ ]:
myenv.docker.base_image
myenv.docker.base_image_registry

You can also specify shared volumes, and shm size.

In [ ]:
myenv.docker.shared_volumes
myenv.docker.shm_size

## Spark and Azure Databricks settings

In addition to Python and Docker settings, Environment also contains attributes for Spark and Azure Databricks runs. These attributes become relevant when you submit runs on those compute targets.

## Next steps

Train with ML frameworks on Azure ML:

* [Train with ML frameworks](../../ml-frameworks)

Learn more about registering and deploying a model:

* [Register Model and deploy as Webservice](../../deployment/deploy-to-cloud/model-register-and-deploy.ipynb)